In [2]:
import sys, os

MARIPOSA_PATH = "/home/zhengyal/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import DbgMode
from debugger.bench_viewer import BenchViewer
from debugger.debugger_options import DebugOptions
from debugger.strainer import DebugStatus
from benchmark_consts import *

/home/zhengyal/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
options = DebugOptions()
options.is_verus = True
verus_bench = BenchViewer(UNSTABLE_VERUS, options)

options = DebugOptions()
options.is_verus = False
mariposa_bench = BenchViewer(UNSTABLE_MARIPOSA, options)

[WARN] [init] 8d058577cd trace slow unknown, fallback to fast_fail 
[WARN] [proof] no proofs available [WARN] [proof] no proofs available 

[WARN] [proof] no proofs available 
[WARN] [init] 0b497bc090 trace did not fail, fallback to fast_fail 
[WARN] [init] e998d08b31 trace slow unknown, fallback to fast_fail 
[WARN] [init] 2a3b1202bd trace slow unknown, fallback to fast_fail 
[WARN] [init] 3cead21983 trace slow unknown, fallback to fast_fail 
[WARN] [proof] no proofs available 
[WARN] [init] 3ce5c5e4dc trace slow unknown, fallback to fast_fail 
[WARN] [init] 5b7d18ab3e trace did not fail, fallback to fast_fail 
[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 
[WARN] [init] d4e5e77cbc trace slow unknown, fallback to fast_fail 
[WARN] [proof] no proofs available 


In [ ]:
# only_inst_fixable = set()

action_to_count = {}
total = 0

for bench_name, bench in [("verus", verus_bench), ("mariposa", mariposa_bench)]:
    for q in bench.status[DebugStatus.FIX_FOUND]:
        stabilized = bench[q].report.stabilized
        mode = bench[q].mode.value
        # print(stabilized)
        for _, edit in stabilized.iterrows():
            total += 1
            k = bench_name, mode, "inst" if edit.action in ["inst_keep", "inst_replace"] else edit.action
            if k not in action_to_count:
                action_to_count[k] = 0
            action_to_count[k] += 1

# print(r"Edit & Mariposa QU & Verus QU & Mariposa TO & Verus TO \\")
# print(r"\hline")

print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(r"\begin{tabular}{l|rrr|rrr}")
print(r"& \multicolumn{3}{c|}{Quick Unknown} & \multicolumn{3}{c}{Slow Timeout} \\")
print(r"\hline")
print(r"Edit & Mariposa & Verus & Total & Mariposa & Verus & Total \\")
print(r"\hline")

total_qu = sum([n for k, n in action_to_count.items() if k[1] == "fast_fail"])
total_to = sum([n for k, n in action_to_count.items() if k[1] == "timeout"])

for i, (action, name) in enumerate((("erase", "Delete"), ("inst", "Instantiate"), ("skolemize", "Skolemize"))):
    mariposa_fast_fail = action_to_count.get(("mariposa", "fast_fail", action), 0)
    verus_fast_fail = action_to_count.get(("verus", "fast_fail", action), 0)
    total_fast_fail = mariposa_fast_fail + verus_fast_fail

    mariposa_timeout = action_to_count.get(("mariposa", "timeout", action), 0)
    verus_timeout = action_to_count.get(("verus", "timeout", action), 0)
    total_timeout = mariposa_timeout + verus_timeout

    print(f"{name} & {mariposa_fast_fail} & {verus_fast_fail} & {round(total_fast_fail / total_qu * 100)}\\% & {mariposa_timeout} & {verus_timeout} & {round(total_timeout / total_to * 100)}\\% \\\\")

print(r"\end{tabular}")
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

print(action_to_count)
print("total queries", total)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
& \multicolumn{3}{c|}{Quick Unknown} & \multicolumn{3}{c}{Slow Timeout} \\
\hline
Edit & Mariposa & Verus & Total & Mariposa & Verus & Total \\
\hline
Delete & 0 & 0 & 0\% & 627 & 34 & 70\% \\
Instantiate & 135 & 36 & 96\% & 222 & 57 & 30\% \\
Skolemize & 8 & 0 & 4\% & 0 & 0 & 0\% \\
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
{('verus', 'fast_fail', 'inst'): 36, ('verus', 'timeout', 'inst'): 57, ('verus', 'timeout', 'erase'): 34, ('mariposa', 'timeout', 'inst'): 222, ('mariposa', 'timeout', 'erase'): 627, ('mariposa', 'fast_fail', 'inst'): 135, ('mariposa', 'fast_fail', 'skolemize'): 8}
total queries 1119
